In [36]:
import numpy as np
import pandas as pd
import random

In [37]:
def readInput(fileName):
    board = []
    row = []
    inputFile = open(fileName, 'r')
    for i in range(9):
        row = []
        line = inputFile.readline()
        line = line.replace('\n', '')
        line = line.replace(' ', '')
        for j in range(9):
            row.append(int(line[j]))
        board.append(row)
    inputFile.close()
    return board

In [38]:
class Constant:
    def __init__(self, input):
        self.initBoard = np.asarray(input)
        self.popSize = 128
        self.chromosomes = self.generateInitialPopulation()
        self.elitismPercentage = 0.50
        self.pc = 0.5
        self.pm = 0.2
        
        
    def set(self):
        self.pm = self.pm - 0.0005
        
    def generateInitialPopulation(self):
        chromosomes = []
        for _ in range(self.popSize):
            chromosome =[]
            for i in range (9):
                shuffle = []
                allNumberRow = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
                possibleNum = list(set(allNumberRow).symmetric_difference(self.initBoard[i]))
                shuffle = random.sample(possibleNum, len(possibleNum))
                for j in range (0, 9):
                    if self.initBoard[i][j] != 0:
                        shuffle.insert(j, self.initBoard[i][j] )
                chromosome.append(shuffle)
            chromosome = np.asarray(chromosome)
            chromosomes.append([chromosome, self.calculateFitness(chromosome)])
        return chromosomes
    
    def calculateFitness(self, chromosome):
        fitness = 0
        df_chromosome = pd.DataFrame(chromosome)
        #     first rule:
        fitness += df_chromosome.apply(lambda x: x.duplicated(), axis = 1).sum().sum()
        #     second rule:
        fitness += df_chromosome.apply(lambda x: x.duplicated(), axis = 0).sum().sum()
        #     third rule:
        fitness += get_fitness_subgrids(chromosome)
        return fitness
    
    def crossOver(self, chromosome1, chromosome2):
        chromosome1 = np.asarray(chromosome1[0])
        chromosome2 = np.asarray(chromosome2[0])
        child_chromosome = []
        for gp1, gp2 in zip(chromosome1, chromosome2):
            prob = random.random()
            if prob < self.pc:
                child_chromosome.append(gp1)
            else:
                child_chromosome.append(gp2)
        
        child_chromosome = np.asarray(child_chromosome)
        return [child_chromosome,self.calculateFitness(child_chromosome)]
        
    def mutate(self, chromosome):
        for row in range (0, 9):
            column1 = random.randrange(0,9)
            column2 = random.randrange(0,9)
            if (self.initBoard[row][column1] == 0) and (self.initBoard[row][column2] == 0):
                temp = chromosome[0][row][column1]
                chromosome[0][row][column1] = chromosome[0][row][column2]
                chromosome[0][row][column2] = temp
        chromosome[1] = self.calculateFitness(chromosome[0])
        return chromosome

    def generateNewPopulation(self):
        new_generation = []
        s = int(self.elitismPercentage*self.popSize)
        for i in range(s):
            child = self.chromosomes[i]
            new_generation.append(child)

        s = self.popSize - s
        for _ in range(s):
            parent1 = random.choice(self.chromosomes)
            parent2 = random.choice(self.chromosomes)
            child= self.crossOver(parent1, parent2)
            prob = random.random()
            if prob < self.pm:
                child = self.mutate(child)
            new_generation.append(child)
        self.chromosomes = new_generation
        return

    
    def solve(self): 
        last = 0
        count = 0
        found = False
        while not found:
            self.chromosomes = sorted(self.chromosomes, key = lambda x:x[1])
            fitness = self.chromosomes[0][1]
            count += 1
            if last > self.chromosomes[0][1] :
                count = 1
                print(self.chromosomes[0][1])
#                 print(self.chromosomes[0][0])
#                 print('_________________')
            if self.chromosomes[0][1] <= 0:
                found = True
                break
            if count > 70:
                self.pm = 0.8
            if count > 200:
                self.chromosomes = self.generateInitialPopulation()
                self.pm = 0.2
            self.generateNewPopulation()
            last = self.chromosomes[0][1]
        print(self.chromosomes[0][0])
        return


In [39]:
def get_fitness_subgrids(grid):
    fitness = 0
    subgrids = []
    for box_i in range(3):
        for box_j in range(3):
            subgrid = []
            for i in range(3):
                for j in range(3):
                    subgrid.append(grid[3*box_i + i][3*box_j + j])
            subgrids.append(subgrid)
    subgrid = np.asarray(subgrids)
    df_subgrid = pd.DataFrame(subgrid)
    fitness += df_subgrid.apply(lambda x: x.duplicated(), axis = 1).sum().sum()
    return fitness 

In [40]:
input = readInput('Test1.txt')  
initConstant = Constant(input)
# chromosoms = initConstant.generateInitialPopulation()
initConstant.solve()

33
32
30


KeyboardInterrupt: 